Swifter digunakan untuk mempersingkat dan memantau proses running sehingga dapat di estimasi perkiraan waktu yang dibutuhkan hingga proses running berakhir

In [ ]:
import swifter

In [ ]:
import numpy as np
import pandas as pd

train = pd.read_csv('train_preprocessed.csv')
test = pd.read_csv('test_preprocessed.csv')
train_hourly = pd.read_csv('train_hourly_preprocessed.csv')
test_hourly = pd.read_csv('test_hourly_preprocessed.csv')

all = [train, test, train_hourly, test_hourly]
for i in all:
  print(i.shape)

(13258, 19)
(4972, 18)
(318192, 16)
(119328, 16)


##**Gain Insight from the Train Hourly and Test Hourly**##

In [ ]:
src = [train_hourly, test_hourly] 

for df in src:
  print(df.columns)

Index(['Unnamed: 0', 'relativehumidity_2m (%)', 'pressure_msl (hPa)',
       'surface_pressure (hPa)', 'cloudcover (%)', 'direct_radiation (W/m²)',
       'diffuse_radiation (W/m²)', 'city', 'year', 'month', 'date',
       'timestamp', 'windspeed', 'winddirection', 'soil_temperature',
       'soil_moisture'],
      dtype='object')
Index(['Unnamed: 0', 'relativehumidity_2m (%)', 'pressure_msl (hPa)',
       'surface_pressure (hPa)', 'cloudcover (%)', 'direct_radiation (W/m²)',
       'diffuse_radiation (W/m²)', 'city', 'year', 'month', 'date',
       'timestamp', 'windspeed', 'winddirection', 'soil_temperature',
       'soil_moisture'],
      dtype='object')


Mengingat fitur year dan month tidak begitu relevan terhadap proses hookup data berdasarkan rata-rata dari tiap city dan date, maka fitur ini akan di drop

In [ ]:
for df in src:
  df.drop(['year', 'month'], axis=1, inplace=True)
  print(df.columns)

Index(['Unnamed: 0', 'relativehumidity_2m (%)', 'pressure_msl (hPa)',
       'surface_pressure (hPa)', 'cloudcover (%)', 'direct_radiation (W/m²)',
       'diffuse_radiation (W/m²)', 'city', 'date', 'timestamp', 'windspeed',
       'winddirection', 'soil_temperature', 'soil_moisture'],
      dtype='object')
Index(['Unnamed: 0', 'relativehumidity_2m (%)', 'pressure_msl (hPa)',
       'surface_pressure (hPa)', 'cloudcover (%)', 'direct_radiation (W/m²)',
       'diffuse_radiation (W/m²)', 'city', 'date', 'timestamp', 'windspeed',
       'winddirection', 'soil_temperature', 'soil_moisture'],
      dtype='object')


In [ ]:
target = ['city', 'date']
for df in src:
  for i in target:
    print(df[i].nunique())

10
1418
10
508


In [ ]:
list(train_hourly.columns)

['Unnamed: 0',
 'relativehumidity_2m (%)',
 'pressure_msl (hPa)',
 'surface_pressure (hPa)',
 'cloudcover (%)',
 'direct_radiation (W/m²)',
 'diffuse_radiation (W/m²)',
 'city',
 'date',
 'timestamp',
 'windspeed',
 'winddirection',
 'soil_temperature',
 'soil_moisture']

In [ ]:
list(test_hourly.columns)

['Unnamed: 0',
 'relativehumidity_2m (%)',
 'pressure_msl (hPa)',
 'surface_pressure (hPa)',
 'cloudcover (%)',
 'direct_radiation (W/m²)',
 'diffuse_radiation (W/m²)',
 'city',
 'date',
 'timestamp',
 'windspeed',
 'winddirection',
 'soil_temperature',
 'soil_moisture']

Pada metode dibawah ini, akan ditentukan nilai-nilai pada suatu feature yang berasal dari dataset dengan **city** dan **date** tertentu. Seperti yang dapat dilihat pada tahapan sebelumnya, jumlah nilai unik dari **city** pada kedua dataset adalah sebanyak 10, dan jumlah nilai unik pada fitur **date** sebanyak 1418 pada training dan 508 pada testing dataset

In [ ]:
obj = list(train_hourly.columns)
obj.remove('Unnamed: 0'), obj.remove('city'), obj.remove('date'), obj.remove('timestamp')

def hookup(dataset, data):
    result = pd.DataFrame()
    for col in obj:
        result[col] = [dataset[(dataset['city']==i) & (dataset['date'] == j)][col].mean() 
                       for i in dataset['city'].unique() 
                       for j in data[data['city']==i]['date'].unique()]
    return result

In [ ]:
def hookup_std(dataset, data):
    result = pd.DataFrame()
    for col in obj:
        result[col] = [dataset[(dataset['city']==i) & (dataset['date'] == j)][col].std() 
                       for i in dataset['city'].unique() 
                       for j in data[data['city']==i]['date'].unique()]
    return result

In [ ]:
ar = hookup(train_hourly, train)
br = hookup(test_hourly, test)

In [ ]:
print(ar.shape, br.shape, ar_std.shape, br_std.shape)

Hasil dari proses hookup yang dilakukan pada train_hourly dan test_hourly akan disimpan dalam bentuk csv

In [ ]:
ar.to_csv('train_final_mean.csv')

In [ ]:
br.to_csv('test_final_mean.csv')

In [ ]:
ar_std.to_csv('train_final_std.csv')

In [ ]:
br_std.to_csv('test_final_std.csv')